In [2]:
from openalgo import api

# Initialize the API client
client = api(api_key='917dd42d55f63ae8f00117abfbe5b05465fc3bd76a3efbee3be7c085df0be579', host='http://127.0.0.1:5000')

# Fetch historical data for SBIN
df = client.history(
    symbol="SBIN",
    exchange="NSE",
    interval="D",
    start_date="2025-01-01",
    end_date="2025-01-24"
)

df

,close,high,low,open,volume
timestamp,,,,,
2025-01-01,793.20,798.65,789.50,795.55,7273098
2025-01-02,801.20,802.40,787.00,795.00,11398319
2025-01-03,793.40,809.50,792.15,801.20,13336514
2025-01-06,776.40,797.60,773.15,797.40,11419367
2025-01-07,778.75,783.85,774.85,779.85,9669152
2025-01-08,771.15,783.95,760.10,780.90,15641490
2025-01-09,760.45,771.35,726.25,771.15,15351029
2025-01-10,743.25,763.40,742.05,760.35,11779296
2025-01-13,729.50,743.95,722.50,741.50,10414733
